In [1]:
## 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# 可以根据需要导入其他库，比如librosa用于音频处理

import os
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset


In [2]:
TrainDir = "Dataset\TRAIN"
TestDir = "Dataset\TEST"
## 请在这里写代码加载我们划分好的TIMIT训练集和测试集

# 定义函数加载数据集
def load_dataset(data_dir, le = None):
    data = []
    labels = []
    print(os.path.exists(data_dir))
    for speaker_dir in os.listdir(data_dir):
        speaker_path = os.path.join(data_dir, speaker_dir)
        for speaker_id in os.listdir(speaker_path):
            speaker_id_path = os.path.join(speaker_path, speaker_id)
            for file_name in os.listdir(speaker_id_path):
                file_path = os.path.join(speaker_id_path, file_name)
                # 在这里你可以处理每个音频文件，例如提取特征并将特征作为数据，将说话者ID作为标签
                # 这里暂时只返回文件路径，具体特征提取需要在后续完成
                data.append(file_path)
                labels.append(speaker_id)
    if le is None:
        le = LabelEncoder()
        le.fit(labels)
    y = le.transform(labels) # back: inverse_transform

    return data, y, le

# 加载训练集和测试集数据
train_data, train_labels, le = load_dataset(TrainDir)
test_data, test_labels, _ = load_dataset(TestDir, le)
print(train_labels)
print(len(set(train_labels)))
for l in train_labels:
    if l == len(set(train_labels)):
        print(l)
# 打印训练集和测试集大小
print("训练集大小:", len(train_data))
print("测试集大小:", len(test_data))


True
True
[ 16  16  16 ... 422 422 422]
462
训练集大小: 3234
测试集大小: 924


In [3]:
n_classes = len(set(train_labels))
n_features = 20

def extract_mfcc(file_path, num_mfcc=20): # (default num_mfcc: 20)
    # 加载音频文件
    audio, sr = librosa.load(file_path, sr=None)
    # 提取MFCC特征
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=num_mfcc)
    # print(mfccs.shape)
    features = [np.mean(mfcc)for mfcc in mfccs]
    # features = [np.mean(mfcc)+np.std(mfcc)for mfcc in mfccs]
    return np.array(features) # mfccs


# 测试MFCC特征提取
sample_mfcc = extract_mfcc(train_data[0])
print("MFCC特征形状:", sample_mfcc.shape)


MFCC特征形状: (20,)


In [4]:
# 加载训练集和测试集数据
train_mfcc = [extract_mfcc(file_path) for file_path in train_data]
test_mfcc = [extract_mfcc(file_path) for file_path in test_data]

print(train_mfcc[0].shape, train_mfcc[30].shape)

# 划分训练集和测试集
# one-hot label: torch.eye(n_classes)[torch.tensor(train_labels)]
X_train, X_val, y_train, y_val = train_test_split(torch.tensor(train_mfcc), torch.tensor(train_labels), test_size=0.2, random_state=42)


(20,) (20,)


C:\Users\weijing\AppData\Local\Temp\ipykernel_12336\314999835.py:9: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:264.)
  X_train, X_val, y_train, y_val = train_test_split(torch.tensor(train_mfcc), torch.tensor(train_labels), test_size=0.2, random_state=42)


In [5]:
# 定义一个简单的神经网络模型
class SpeakerNet(nn.Module):
    def __init__(self, n_features, n_classes):
        super(SpeakerNet, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(n_features, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(64, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(p=0.2),
            nn.Linear(256, n_classes)
        )

    def forward(self, x):
        return nn.functional.softmax(self.net(x))

# 定义数据集类
class SpeakerDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# 实例化数据集
train_dataset = SpeakerDataset(X_train, y_train)
val_dataset = SpeakerDataset(X_val, y_val)

# 实例化数据加载器
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# 定义模型、损失函数和优化器
model = SpeakerNet(n_features, n_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [6]:
epochs = 200

# 训练模型
for epoch in range(epochs):  # 进行多个训练周期
    model.train()
    for batch_data, batch_labels in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels.long())
        loss.backward()
        optimizer.step()

    # 验证模型
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for batch_data, batch_labels in val_loader:
            outputs = model(batch_data)
            _, predicted = torch.max(outputs.data, 1)
            total += batch_labels.size(0)
            # print(predicted.shape, batch_labels.shape)
            correct += (predicted == batch_labels).sum().item()

        print(f'Epoch [{epoch+1}/{epochs}], Accuracy: {100 * correct / total}%')

# 保存模型
torch.save(model.state_dict(), 'speaker_model.pth')

C:\Users\weijing\AppData\Local\Temp\ipykernel_12336\2567558568.py:18: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return nn.functional.softmax(self.net(x))


Epoch [1/200], Accuracy: 2.7820710973724885%
Epoch [2/200], Accuracy: 3.554868624420402%
Epoch [3/200], Accuracy: 7.882534775888717%
Epoch [4/200], Accuracy: 13.601236476043276%
Epoch [5/200], Accuracy: 19.474497681607417%
Epoch [6/200], Accuracy: 25.038639876352395%
Epoch [7/200], Accuracy: 29.21174652241113%
Epoch [8/200], Accuracy: 35.394126738794434%
Epoch [9/200], Accuracy: 38.639876352395675%
Epoch [10/200], Accuracy: 40.49459041731066%
Epoch [11/200], Accuracy: 41.57650695517774%
Epoch [12/200], Accuracy: 46.21329211746522%
Epoch [13/200], Accuracy: 44.35857805255023%
Epoch [14/200], Accuracy: 47.14064914992272%
Epoch [15/200], Accuracy: 49.76816074188562%
Epoch [16/200], Accuracy: 49.92272024729521%
Epoch [17/200], Accuracy: 51.62287480680062%
Epoch [18/200], Accuracy: 51.46831530139104%
Epoch [19/200], Accuracy: 51.15919629057187%
Epoch [20/200], Accuracy: 53.013910355486864%
Epoch [21/200], Accuracy: 53.32302936630603%
Epoch [22/200], Accuracy: 55.95054095826893%
Epoch [23/20

Epoch [182/200], Accuracy: 71.4064914992272%
Epoch [183/200], Accuracy: 70.17001545595055%
Epoch [184/200], Accuracy: 72.02472952086553%
Epoch [185/200], Accuracy: 72.64296754250387%
Epoch [186/200], Accuracy: 71.25193199381762%
Epoch [187/200], Accuracy: 71.56105100463678%
Epoch [188/200], Accuracy: 72.17928902627511%
Epoch [189/200], Accuracy: 68.31530139103555%
Epoch [190/200], Accuracy: 70.17001545595055%
Epoch [191/200], Accuracy: 72.02472952086553%
Epoch [192/200], Accuracy: 70.17001545595055%
Epoch [193/200], Accuracy: 71.25193199381762%
Epoch [194/200], Accuracy: 70.47913446676971%
Epoch [195/200], Accuracy: 70.17001545595055%
Epoch [196/200], Accuracy: 72.79752704791345%
Epoch [197/200], Accuracy: 69.24265842349304%
Epoch [198/200], Accuracy: 69.55177743431221%
Epoch [199/200], Accuracy: 69.7063369397218%
Epoch [200/200], Accuracy: 70.01545595054095%
